# Actividad 3: Modelado Documental

**Asignatura:** Big Data Aplicado.
**Estudiante:** Byron V. Blatch Rodríguez.
**Fecha:** 25/11/2025

## Objetivos de la actividad
1. Generar un modelo documental del dataset que se ha trabajado en las actividades anteriores.

> Nota: El modelado documental implica pasar de un paradigma relacional (Tablas/Excel/CSV) al paradigma NoSQL (Documentos/JSON/BSON).
---

En un Modelo Documental (como MongoDB), los datos son jerárquicos. No se repite "Venezuela" 50 veces como sucede en `vnzla_population_limpio.csv`. Se crea UN documento que contiene toda la "historia". Por tanto, transformaremos los datos planos `CSV` a una estructura `JSON` anidada (Documento) listo para ser insertado en una base de datos NoSQL. Para ello escribiremos un script que haga esta transformación.

In [15]:
# Importamos las librerías que utilizaremos.
import pandas as pd
import json
import datetime

# Cargamos el dataset limpio generado en la actividad anterior.
df = pd.read_csv('data/vnzla_population_limpio.csv')

# Agregamos las columnas de Latitud y Longitud para Venezuela que habíamos determinado en la actividad anterior.
df['lat'] = 6.42375
df['lon'] = -66.58973

# Comenzamos a construir el documento.
# Primero, obtenemos los datos generales de Venezuela.

info_pais = df.iloc[0]

pais_nombre = info_pais['Country/Territory']
pais_lat = info_pais['lat']
pais_lon = info_pais['lon']

# Ahora necesitamos transformar las filas planas en una lista de objetos anidados.
historial = [] # Creamos una lista vacía para almacenar los pedacitos de historia.

# Lo siguiente creará un bucle que recorra cada fila del DataFrame.
for index, row in df.iterrows(): # index es el número de la fila, row es la fila en sí.
    registro = { 
        "Year": int(row['Year']),
        "Population": int(row['Population'])
        } # En cada iteración, creamos un diccionario (mini-objeto JSON) con los datos de esa fila y lo añadimos a la lista historial.
    historial.append(registro) # Coge el mini-objeto JSON y lo añade a la lista historial.

# Finalmente, construimos el documento completo ya que tenemos las variables sueltas y la lista historial completa.

documento_pais = {
    "pais": pais_nombre, # Nivel 1: String simple
    "geografia": {      # Nivel 2: Objeto anidado
        "tipo": "Punto",
        "coordenadas": [pais_lat, pais_lon] # Lista ordenada [Latitud, Longitud]
    },
    "datos_historicos": historial, # Nivel 3: Lista compleja que generamos en la iteración.
    "metadata": {
        "creado_el": datetime.datetime.now().isoformat(),
        "autor": "Byron V. Blatch Rodríguez"
    }
}

Si nos fijamos en la estructura, no es plana:
- Tiene profundidad. `geografía` tiene cosas. `datos_historicos` contiene la lista.
- Es un modelo documental. Agrupa la información por contexto, no por filas.
- En el caso de `isoformat()`, convierte la fecha actual en un string estándar internacional legible por cualquier sistema.

In [16]:
# Serializamos el documento a formato JSON y lo guardamos en un archivo.
print("Generando el archivo JSON del modelo documental...")
print(json.dumps(documento_pais, indent=4)) # Imprime el JSON en pantalla con indentación para mejor lectura.

Generando el archivo JSON del modelo documental...
{
    "pais": "Venezuela",
    "geografia": {
        "tipo": "Punto",
        "coordenadas": [
            6.42375,
            -66.58973
        ]
    },
    "datos_historicos": [
        {
            "Year": 2000,
            "Population": 24427729
        },
        {
            "Year": 2010,
            "Population": 28715022
        },
        {
            "Year": 2015,
            "Population": 30529716
        },
        {
            "Year": 2020,
            "Population": 28490453
        },
        {
            "Year": 2022,
            "Population": 28301696
        }
    ],
    "metadata": {
        "creado_el": "2025-11-25T17:21:07.114983",
        "autor": "Byron V. Blatch Rodr\u00edguez"
    }
}


Para finalizar la actividad, exportamos nuestro `JSON` con el siguiente código.


In [17]:
# Defino la ruta donde se guardará el archivo JSON
ruta_salida = 'data/vnzla_population_modelo_documental.json'

# Abro el archivo en modo escritura ('w') y guardo el JSON
with open(ruta_salida, 'w') as f:
    # Volcamos el diccionario del archivo con formato bonito
    json.dump(documento_pais, f, indent=4)
print(f"Archivo JSON guardado exitosamente en: {ruta_salida}")

Archivo JSON guardado exitosamente en: data/vnzla_population_modelo_documental.json
